In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)


(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [ ]:
Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'lbfgs'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [8]:
# Initialize MLflow
mlflow.set_experiment("deployment model")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/11/04 19:29:41 INFO mlflow.tracking.fluent: Experiment with name 'deployment model' does not exist. Creating a new experiment.
2025/11/04 19:29:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 19:29:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:29:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/132371154868234300/runs/ba6accb41a0149fd8e07c415ffa6075b
🧪 View experiment at: http://localhost:5000/#/experiments/132371154868234300


2025/11/04 19:30:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:30:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/132371154868234300/runs/684f1f4fc20249bfb7deae7e94700bcf
🧪 View experiment at: http://localhost:5000/#/experiments/132371154868234300


2025/11/04 19:30:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/04 19:30:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/132371154868234300/runs/ce322d05de0a4a8db706c7ede4bc31ea
🧪 View experiment at: http://localhost:5000/#/experiments/132371154868234300


2025/11/04 19:30:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/132371154868234300/runs/ea8b813fe1904082934cc89e2d722142
🧪 View experiment at: http://localhost:5000/#/experiments/132371154868234300


Register the Model

In [2]:
import mlflow

In [5]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: file:///Users/manishuxuid/Desktop/FSDS/FSDS%20Notes/November%20Classes%20notes/MLOPS/mlruns


In [6]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5001")

model_name = "XGB-Smote"
run_id = "ea8b813fe1904082934cc89e2d722142"
model_uri = f"runs:/{run_id}/model"

try:
    result = mlflow.register_model(model_uri=model_uri, name=model_name)
    print(f"✅ Model '{model_name}' registered successfully (version: {result.version})")
except Exception as e:
    print("❌ Error while registering model:", e)


Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/11/04 19:54:47 WARNING mlflow.tracking._model_registry.fluent: Run with id ea8b813fe1904082934cc89e2d722142 has no artifacts at artifact path 'model', registering model based on models:/m-e5d0424d1847490b9137f37400a2d71c instead
2025/11/04 19:54:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1
Created version '1' of model 'XGB-Smote'.


✅ Model 'XGB-Smote' registered successfully (version: 1)
